In [1]:
# !pip install SoccerNet
# !pip install boto3
# !pip install imageio
# !pip install imageio[pyav]

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/10.8 MB 49.2 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 2.2/3.4 MB 44.4 MB/s eta 0:00:01
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 94, in read
    self.__buf.write(data)
  File "/usr/lib/python3.8/tempfile.py", line 612, in func_wrapper
    return func(*args, **kwargs)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurre

In [14]:
import numpy as np
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import zipfile
from PIL import Image
from tqdm import tqdm
import os
import json
from SoccerNet.Evaluation.utils import FRAME_CLASS_DICTIONARY
import copy
import time
from SoccerNet.utils import getListGames
from argparse import ArgumentParser
import boto3
import io
import imageio
import config
# import skvideo.io  
# import cv2
# from PIL import image
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

In [2]:
s3 = boto3.resource('s3')
session = boto3.Session(
    aws_access_key_id=config.aws_access_key_id,
    aws_secret_access_key=config.aws_secret_access_key,
)
s3 = session.resource('s3')

bucket = s3.Bucket('soccernet-230')
keys = [obj.key for obj in bucket.objects.all()]
print(keys[:5])
  
    

['caption-2023/', 'caption-2023/.DS_Store', 'caption-2023/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_baidu_soccer_embeddings.npy', 'caption-2023/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/2_baidu_soccer_embeddings.npy', 'caption-2023/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/Labels-caption.json']


In [3]:
obj = s3.Object('soccernet-230', key='caption-2023/italy_serie-a/2016-2017/2017-04-09 - 21-45 Lazio 0 - 3 Napoli/Labels-caption.json')
print(obj.get())

{'ResponseMetadata': {'RequestId': 'SMQ8H10X3GYE4TW0', 'HostId': 'OreXe6gQ+6KNe5sgyLs/I6hCutNxmQmCU3SVATFUhFDauPiJKb+aOAFjtdQc9UmaPkMQEwGu334=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'OreXe6gQ+6KNe5sgyLs/I6hCutNxmQmCU3SVATFUhFDauPiJKb+aOAFjtdQc9UmaPkMQEwGu334=', 'x-amz-request-id': 'SMQ8H10X3GYE4TW0', 'date': 'Thu, 25 May 2023 19:43:35 GMT', 'last-modified': 'Fri, 19 May 2023 17:33:28 GMT', 'etag': '"e3b3353efe74e55e36ab5befab50a0f9"', 'x-amz-server-side-encryption': 'AES256', 'accept-ranges': 'bytes', 'content-type': 'application/json', 'server': 'AmazonS3', 'content-length': '65293'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2023, 5, 19, 17, 33, 28, tzinfo=tzutc()), 'ContentLength': 65293, 'ETag': '"e3b3353efe74e55e36ab5befab50a0f9"', 'ContentType': 'application/json', 'ServerSideEncryption': 'AES256', 'Metadata': {}, 'Body': <botocore.response.StreamingBody object at 0x7f67c30c9400>}


In [4]:
obj_reference = s3.Object('soccernet-230', 'caption-2023/italy_serie-a/2016-2017/2017-04-09 - 21-45 Lazio 0 - 3 Napoli/Labels-caption.json')
body = obj_reference.get()['Body'].read().decode('utf-8')
body = json.loads(body)
print(body['annotations'][1])

{'important': True, 'gameTime': '2 - 46:56', 'label': 'soccer-ball', 'description': 'Goal - 0:3! Piotr Zielinski laid the ball off to Lorenzo Insigne (Napoli), who shot from close range into the bottom left corner, leaving the goalkeeper helpless.', 'identified': 'Goal - 0:3! [PLAYER_dlFNZSvb] laid the ball off to [PLAYER_6qUE0AeN] ([TEAM_]), who shot from close range into the bottom left corner, leaving the goalkeeper helpless.', 'anonymized': 'Goal - 0:3! [PLAYER] laid the ball off to [PLAYER] ([TEAM]), who shot from close range into the bottom left corner, leaving the goalkeeper helpless.', 'visibility': 'shown', 'position': '2816000'}


In [5]:
print(body.keys())

dict_keys(['timestamp', 'score', 'round', 'teams', 'lineup', 'referee', 'venue', 'attendance', 'referee_matched', 'referee_found', 'coach_matched', 'gameHomeTeam', 'home', 'gameAwayTeam', 'away', 'gameDate', 'annotations'])


In [8]:
list_games = getListGames(split='valid', task="caption")
print(list_games[:5])

['england_epl/2014-2015/2015-04-11 - 19-30 Burnley 0 - 1 Arsenal', 'england_epl/2015-2016/2015-08-30 - 18-00 Swansea 2 - 1 Manchester United', 'england_epl/2015-2016/2015-09-26 - 17-00 Leicester 2 - 5 Arsenal', 'england_epl/2015-2016/2015-09-26 - 17-00 Manchester United 3 - 0 Sunderland', 'england_epl/2015-2016/2015-10-03 - 17-00 Manchester City 6 - 1 Newcastle Utd']


In [21]:
class SNV3Dataset(Dataset):
  def __init__(self, bucket='soccernet-230', split='train', resolution=(480,360)):

    # Initialize s3 resource
    s3 = boto3.resource('s3')
    self.session = boto3.Session(
        aws_access_key_id=config.aws_access_key_id,
        aws_secret_access_key=config.aws_secret_access_key,
    )
    self.s3 = session.resource('s3')
    # self.client = boto3.client('s3')
    # self.bucket = s3.Bucket(bucket)
    self.bucket = bucket

    # Get list of selected subset of games
    self.list_games = getListGames(split, task="frames")

    self.resolution = resolution
    self.resize = torchvision.transforms.Resize((resolution[1],resolution[0]), antialias=True)
    # self.preload_images = preload_images
    # self.zipped_images = zipped_images

    print("Reading the annotation files")
    self.metadata = list()
    for game in tqdm(self.list_games):
      # self.metadata.append(json.load(open(os.path.join(self.path, game, "Labels-v3.json"))))
      # print(f'getting key: {game}')
      try:
        obj_reference = self.s3.Object('soccernet-230', 'caption-2023/' + game + '/Labels-caption.json')
        obj = json.loads(obj_reference.get()['Body'].read().decode('utf-8'))
        obj['game'] = game
        self.metadata.append(obj)
      except:
        print(f'Cannot find game: {game}')
        continue

    self.data = list()
    for label in tqdm(self.metadata):
      # Retrieve each action in the game
      for annotation in label["annotations"]:
        # concatenate the replays of each action with itself
        annotation['game'] = label['game']
        self.data.append(annotation)

  def __getitem__(self, index):
    print('IN GET_ITEM')
    label = self.data[index]
    embed_ref = self.s3.Object('soccernet-230', 'caption-2023/' + label['game'] + '/1_baidu_soccer_embeddings.npy')
    vid_ref = self.s3.Object('soccernet-230', 'videos/' + label['game'] + '/1_224p.mkv')
    
    with io.BytesIO(embed_ref.get()["Body"].read()) as e_f:
      e_f.seek(0)  # rewind the file
      embed = np.load(e_f, allow_pickle=True)
      print(embed.shape)
    with io.BytesIO(vid_ref.get()["Body"].read()) as v_f:
      v_f.seek(0)  # rewind the file
      video = v_f.read()

      idxs = range(0, 10)
      print('pre-imread')
      video_list = [imageio.v3.imread(video, format_hint='.mkv', index=idx, plugin='pyav') for idx in idxs]
      print('post-imread')
      video = np.stack(video_list)
      print(video.shape)
      
      # # im = Image.open(v_f)
      # # print(np.array(im))
      # bucket = self.s3.Bucket('caption-2023')
      # print('test1')
      # obj = bucket.Object('soccernet-230', 'videos/' + label['game'] + '/1_224p.mkv')
      # print('test2')
      # file_stream = io.BytesIO()
      # print('test3')
      # obj.download_fileobj(file_stream)
      # print('test4')
      # video = mpimg.imread(file_stream)
      # print(video)

      # vid = np.load(v_f, allow_pickle=True)

    # video = skvideo.io.vread(v_f)  
    # print(video.shape)
    
    out = {
      'label' : label,
      'embedding' : embed,
      'video' : video
    }
    print('RETURNING...')
    print(out)
    return out

In [22]:
test_ds = SNV3Dataset(split='test',
                      bucket='soccernet-230')

Reading the annotation files


 49%|█████████████████████████████████████████████████████████████████▊                                                                    | 27/55 [00:01<00:01, 23.74it/s]

Cannot find game: france_ligue-1/2016-2017/2016-11-30 - 23-00 Paris SG 2 - 0 Angers


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 58661.59it/s]


In [23]:
asdf = None
for example in test_ds:
  # asdf = example
  print(example['video'].shape)
  break

IN GET_ITEM
CREATED OBJECT
(2700, 8576)
pre-imread
post-imread
(10, 224, 398, 3)
RETURNING...
{'label': {'important': False, 'gameTime': '2 - 50:59', 'label': '', 'description': 'We saw a few chances and nice plays, but the overall impression of the game is uninspiring and not hugely entertaining. A very even game was seen on the pitch today. Both sides tried to combine well and to keep the ball at their feet.', 'identified': 'We saw a few chances and nice plays, but the overall impression of the game is uninspiring and not hugely entertaining. A very even game was seen on the pitch today. Both sides tried to combine well and to keep the ball at their feet.', 'anonymized': 'We saw a few chances and nice plays, but the overall impression of the game is uninspiring and not hugely entertaining. A very even game was seen on the pitch today. Both sides tried to combine well and to keep the ball at their feet.', 'visibility': 'shown', 'position': '3059000', 'game': 'england_epl/2014-2015/201

In [ ]:
print(asdf)

In [ ]:
class SNV3Dataset(Dataset):

	def __init__(self, path, split="all", resolution=(1920,1080), preload_images=False, tiny=None, zipped_images=False):

		# Path for the SoccerNet-v3 dataset 
		# containing the images and labels
		self.path = path

		# Get the list of the selected subset of games
		self.list_games = getListGames(split, task="frames")
		if tiny is not None:
			self.list_games = self.list_games[:tiny]

		# Resolution of the images to load (width, height)
		self.resolution = resolution
		self.resize = torchvision.transforms.Resize((resolution[1],resolution[0]), antialias=True)
		self.preload_images = preload_images
		self.zipped_images = zipped_images

		# Variable to store the metadata
		print("Reading the annotation files")
		self.metadata = list()
		for game in tqdm(self.list_games):
			self.metadata.append(json.load(open(os.path.join(self.path, game, "Labels-v3.json"))))

		# Variables to store the preloaded images and annotations
		# Each element in the list is a list of images and annotations linked to an action
		self.data = list()
		for annotations in tqdm(self.metadata):

			# Retrieve each action in the game
			for action_name in annotations["GameMetadata"]["list_actions"]:

				# concatenate the replays of each action with itself
				img_list = [action_name] + annotations["actions"][action_name]["linked_replays"]
				self.data.append(list())
				IDs_list = list()

				zipfilepath = os.path.join(self.path,annotations["GameMetadata"]["UrlLocal"], 'Frames-v3.zip')
				if self.zipped_images:
					zippedFrames = zipfile.ZipFile(zipfilepath, 'r')

				# For each image extract the images and annotations
				for i, img in enumerate(img_list):

					# Variable to save the annotation
					data_tmp = dict()
					data_tmp["image"] = None

					# Only the first frame is an action, the rest are replays
					img_type = "actions"
					if i > 0 :
						img_type="replays"

					filepath = os.path.join(self.path,annotations["GameMetadata"]["UrlLocal"], "v3_frames", img)
					if self.preload_images:
						with torch.no_grad():
							if self.zipped_images:
								imginfo = zippedFrames.open(img)
								data_tmp["image"] = self.resize(transforms.ToTensor()(Image.open(imginfo))*255)
							else:
								data_tmp["image"] = self.resize(torchvision.io.read_image(filepath))

					data_tmp["zipfilepath"] = zipfilepath
					data_tmp["imagefilepath"] = img	
					data_tmp["filepath"] = filepath					
				
					data_tmp["bboxes"], ID_tmp = self.format_bboxes(annotations[img_type][img]["bboxes"], annotations[img_type][img]["imageMetadata"])
					
					data_tmp["lines"] = self.format_lines(annotations[img_type][img]["lines"], annotations[img_type][img]["imageMetadata"])
					
					data_tmp["links"] = None

					IDs_list.append(ID_tmp)

					self.data[-1].append(data_tmp)

				self.format_links(IDs_list)

	def format_bboxes(self, bboxes, image_metadata):

		# Bounding boxes in x_top, y_top, width, height, cls_idx, num_idx
		data = list()

		IDs = list()

		for i, bbox in enumerate(bboxes):

			if bbox["class"] is not None:

				tmp_data = torch.zeros((4+1+1,), dtype=torch.float)-1
				tmp_data[0] = bbox["points"]["x1"]/image_metadata["width"]	
				tmp_data[1] = bbox["points"]["y1"]/image_metadata["height"]
				tmp_data[2] = abs(bbox["points"]["x2"]-bbox["points"]["x1"])/image_metadata["width"]	
				tmp_data[3] = abs(bbox["points"]["y2"]-bbox["points"]["y1"])/image_metadata["height"]
				tmp_data[4] = float(FRAME_CLASS_DICTIONARY[bbox["class"]])
				if bbox["ID"] is not None: 
					if bbox["ID"].isnumeric():
						tmp_data[5] = float(bbox["ID"])
				IDs.append([bbox["ID"],FRAME_CLASS_DICTIONARY[bbox["class"]]])
				data.append(tmp_data)

		data = torch.stack(data)
		return data, IDs

	def format_lines(self, lines, image_metadata):

		# Each element is a list with list of points, cls_idx
		data = list()

		for line in lines:

			if line["class"] is not None:
				points = torch.FloatTensor(line["points"])
				points[::2] = points[::2]/image_metadata["width"]
				points[1::2] = points[1::2]/image_metadata["height"]
				data.append([points,torch.FloatTensor([FRAME_CLASS_DICTIONARY[line["class"]]])])
		return data

	def format_links(self, IDs_list):

		# Links are stored as (index of the current image, 
		# index of the bounding box in the first image,
		# index of the second image, 
		# index of the bounding box in the second image)

		for i, IDs_1 in enumerate(IDs_list):

			list_of_links = list()

			for j, IDs_2 in enumerate(IDs_list):

				if i == j:
					continue

				for k, ID_1 in enumerate(IDs_1):

					for l, ID_2 in enumerate(IDs_2):
						if ID_1[1] == ID_2[1]:
							if ID_1[0] is not None and ID_2[0] is not None:
								if ID_1[0] == ID_2[0]:
									list_of_links.append([i,k,j,l])
									continue
			
			self.data[-1][i]["links"] = torch.FloatTensor(list_of_links)


	def __getitem__(self, index):

		if not self.preload_images:
			data = copy.deepcopy(self.data[index])
			with torch.no_grad():
				image_list = list()
				for i, d in enumerate(data):
					if self.zipped_images:
						imginfo = zipfile.ZipFile(d["zipfilepath"], 'r').open(d["imagefilepath"])
						img = transforms.ToTensor()(Image.open(imginfo))*255
						data[i]["image"] = self.resize(img)
			
					else:
						data[i]["image"] = self.resize(torchvision.io.read_image(d["filepath"]))
				return data

		return self.data[index]

	def __len__(self):

		return len(self.data)

if __name__ == "__main__":

	# Load the arguments
	parser = ArgumentParser(description='dataloader')
	
	parser.add_argument('--SoccerNet_path',   required=True, type=str, help='Path to the SoccerNet-V3 dataset folder' )
	parser.add_argument('--tiny',   required=False, type=int, default=None, help='Select a subset of x games' )
	parser.add_argument('--split',   required=False, type=str, default="all", help='Select the split of data' )
	parser.add_argument('--num_workers',   required=False, type=int, default=4, help='number of workers for the dataloader' )
	parser.add_argument('--resolution_width',   required=False, type=int, default=1920, help='width resolution of the images' )
	parser.add_argument('--resolution_height',   required=False, type=int, default=1080, help='height resolution of the images' )
	parser.add_argument('--preload_images', action='store_true', help="Preload the images when constructing the dataset")
	parser.add_argument('--zipped_images', action='store_true', help="Read images from zipped folder")

	args = parser.parse_args()

	
	start_time = time.time()
	soccernet = SNV3Dataset(args.SoccerNet_path, split=args.split, resolution=(args.resolution_width,args.resolution_height), preload_images=args.preload_images, zipped_images=args.zipped_images, tiny=args.tiny)
	soccernet_loader = DataLoader(soccernet, batch_size=1, shuffle=False, num_workers=args.num_workers, pin_memory=True)
	with tqdm(enumerate(soccernet_loader), total=len(soccernet_loader), ncols=160) as t:
		for i, data in t: 
			continue
	end_time = time.time()
	print(end_time-start_time)